In [2]:
import pandas as pd
import random
import os
import requests
import csv
import time
from datetime import datetime

# --- Cấu hình Moodle API ---
MOODLE_URL = 'http://localhost:8100/webservice/rest/server.php'
TOKEN = 'ac6bc02c96ff699f2d0df05b9382e23b' # Thay thế bằng token Moodle của bạn
FORMAT = 'json'

# --- Hằng số và siêu tham số (Hyperparameters) cho Q-learning ---
LEARNING_RATE = 0.1     # Tốc độ học của thuật toán
DISCOUNT_FACTOR = 0.9   # Tầm quan trọng của phần thưởng trong tương lai
EXPLORATION_RATE = 0.2 # Tỷ lệ khám phá (chọn hành động ngẫu nhiên)

# Định nghĩa các hành động (Actions)
ACTIONS = [
    'read_new_resource',
    'review_old_resource',
    'attempt_new_quiz',
    'redo_failed_quiz',
    'skip_to_next_module'
]

# Khởi tạo Q-table (sẽ được tải hoặc huấn luyện)
q_table = {}

# Các bins cho điểm quiz (1-10)
QUIZ_SCORE_BINS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Các bins cho tỷ lệ đọc tài liệu (từ 0.2 đến 1.0)
READ_RATE_BINS = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# --- Hàm gọi Moodle API ---
def call_api(function, extra_params):
    """
    Gọi Moodle Web Service API.
    """
    params = {
        'wstoken': TOKEN,
        'moodlewsrestformat': FORMAT,
        'wsfunction': function
    }
    params.update(extra_params)
    try:
        response = requests.post(MOODLE_URL, data=params)
        response.raise_for_status() # Kiểm tra lỗi HTTP
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi khi gọi Moodle API '{function}': {e}")
        return {} # Trả về dictionary rỗng để tránh lỗi

# --- Hàm lấy Cluster của người dùng từ file CSV ---
def get_user_cluster(user_id):
    """
    Lấy thông tin cluster của người dùng từ file CSV.
    """
    file_path = "./synthetic_user_features_clustered.csv"
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print("❌ Không tìm thấy file: synthetic_user_features_clustered.csv")
        return None
    user_row = df[df['userid'] == user_id]
    if user_row.empty:
        return None
    return user_row.iloc[0]['cluster']

# Ánh xạ loại hành động từ CSV sang các hành động đã định nghĩa
def get_action_from_type(activity_type):
    """
    Ánh xạ loại hoạt động từ log sang hành động Q-learning.
    Lưu ý: Cần logic phức tạp hơn để phân biệt attempt_new_quiz và redo_failed_quiz.
    """
    if activity_type == 'quiz':
        return 'attempt_new_quiz'
    elif activity_type == 'resource' or activity_type == 'hvp':
        return 'read_new_resource'
    else:
        return 'skip_to_next_module'

def discretize_state(avg_quiz_score, completion_rate, quiz_passed, cluster, sectionid):
    """
    Rời rạc hóa state, có thêm sectionid trong trạng thái.
    """
    quiz_bin = 0
    for i, bin_val in enumerate(QUIZ_SCORE_BINS):
        if avg_quiz_score >= bin_val:
            quiz_bin = i + 1
        else:
            break
    
    read_bin = 0
    for i, bin_val in enumerate(READ_RATE_BINS):
        if completion_rate >= bin_val:
            read_bin = i + 1
        else:
            break

    cluster_val = int(cluster) if cluster is not None else 0

    return (quiz_bin, read_bin, int(quiz_passed), cluster_val, int(sectionid))

# --- Hàm lấy giá trị Q-value ---
def get_q_value(state, action):
    """
    Lấy Q-value cho một cặp (state, action). Khởi tạo nếu chưa có.
    """
    global q_table # Dùng biến toàn cục
    if state not in q_table:
        q_table[state] = {a: 0.0 for a in ACTIONS}
    return q_table[state][action]

# --- Hàm cập nhật Q-table ---
def update_q_table(state, action, reward, next_state):
    """
    Cập nhật Q-value theo công thức Q-learning.
    """
    global q_table # Dùng biến toàn cục
    current_q = get_q_value(state, action)
    
    if next_state not in q_table:
        max_future_q = 0.0
    else:
        max_future_q = max(q_table[next_state].values())

    new_q = current_q + LEARNING_RATE * (reward + DISCOUNT_FACTOR * max_future_q - current_q)
    q_table[state][action] = new_q

def get_reward(current_state, action, next_state):
    reward = 0
    print(f"⚙️ Tính reward cho hành động '{action}':")
    print(f"  Current state: {current_state}")
    print(f"  Next state: {next_state}")

    if current_state[2] == 0 and next_state[2] == 1:
        reward += 10
        print("  +10: Quiz passed (từ 0 -> 1)")
    
    if next_state[1] > current_state[1]:
        reward += 5
        print(f"  +5: Completion rate tăng ({current_state[1]} -> {next_state[1]})")
    
    if next_state[0] > current_state[0]:
        reward += 7
        print(f"  +7: Quiz score bin tăng ({current_state[0]} -> {next_state[0]})")
    
    if next_state[0] < current_state[0]:
        reward -= 5
        print(f"  -5: Quiz score bin giảm ({current_state[0]} -> {next_state[0]})")
    
    print(f"👉 Tổng reward: {reward}\n")
    return reward

# --- Hàm lưu Q-table vào file CSV ---
def save_q_table_to_csv(q_table_to_save, filename='q_table_results.csv'):
    """
    Lưu Q-table đã học vào file CSV.
    """
    q_table_data = []
    for state, actions_dict in q_table_to_save.items():
        for action, q_value in actions_dict.items():
            row = {
                'quiz_bin': state[0],
                'read_bin': state[1],
                'quiz_passed': state[2],
                'cluster': state[3],
                'sectionid': state[4],
                'action': action,
                'q_value': q_value
            }
            q_table_data.append(row)
    
    df = pd.DataFrame(q_table_data)
    df.to_csv(filename, index=False)
    print(f"✅ Q-table đã được lưu vào file '{filename}' thành công.")

# --- Hàm tải Q-table từ file CSV ---
def load_q_table_from_csv(filename='q_table_results.csv'):
    """
    Tải Q-table từ file CSV đã lưu.
    """
    q_table_loaded = {}
    if not os.path.exists(filename):
        print(f"❌ Không tìm thấy file '{filename}'. Vui lòng huấn luyện mô hình trước.")
        return {}
    
    df = pd.read_csv(filename)
    for _, row in df.iterrows():
        state = (row['quiz_bin'], row['read_bin'], row['quiz_passed'], row['cluster'], row['sectionid'])
        action = row['action']
        q_value = row['q_value']
        
        if state not in q_table_loaded:
            q_table_loaded[state] = {a: 0.0 for a in ACTIONS}
        
        q_table_loaded[state][action] = q_value
    print(f"✅ Q-table đã được tải từ file '{filename}' thành công.")
    return q_table_loaded

# --- HÀM MỚI: Khởi tạo Q-table với các giá trị ban đầu (0) ---
def initialize_q_table_with_clusters(filename='q_table_results.csv', cluster_file='./synthetic_user_features_clustered.csv'):
    """
    Khởi tạo Q-table với tất cả các trạng thái có thể có và Q-value bằng 0.
    """
    print("✨ Đang khởi tạo Q-table ban đầu...")
    initial_q_table = {}

    try:
        df_clusters = pd.read_csv(cluster_file)
        unique_clusters = df_clusters['cluster'].unique()
    except FileNotFoundError:
        print(f"❌ Không tìm thấy file cluster: {cluster_file}. Không thể khởi tạo Q-table đầy đủ.")
        unique_clusters = [0]

    quiz_bins_range = range(len(QUIZ_SCORE_BINS) + 1)
    read_bins_range = range(len(READ_RATE_BINS) + 1)
    quiz_passed_states = [0, 1]

    if os.path.exists('./user_insight.csv'):
        df_insight = pd.read_csv('./user_insight.csv')
        unique_sectionids = df_insight['sectionid'].unique()
    else:
        unique_sectionids = [1, 2, 3, 4, 5]

    for q_bin in quiz_bins_range:
        for r_bin in read_bins_range:
            for q_passed in quiz_passed_states:
                for cluster in unique_clusters:
                    for section_id in unique_sectionids:
                        state = (q_bin, r_bin, q_passed, int(cluster), int(section_id))
                        initial_q_table[state] = {action: 0.0 for action in ACTIONS}
    
    save_q_table_to_csv(initial_q_table, filename)
    print("✅ Q-table ban đầu đã được tạo và lưu thành công.")
    return initial_q_table

# --- Hàm huấn luyện mô hình từ dữ liệu lịch sử ---
def train_from_csv(file_path):
    """
    Huấn luyện Q-table từ dữ liệu lịch sử trong file CSV.
    """
    global q_table
    print("🚀 Bắt đầu huấn luyện mô hình Q-learning...")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"❌ Không tìm thấy file dữ liệu lịch sử: {file_path}. Không thể huấn luyện.")
        return

    df = df.sort_values(by=['userid', 'time'])

    for userid in df['userid'].unique():
        user_data = df[df['userid'] == userid].reset_index(drop=True)
        for i in range(len(user_data) - 1):
            current_row = user_data.iloc[i]
            next_row = user_data.iloc[i+1]
            
            if current_row['sectionid'] != next_row['sectionid']:
                continue

            state = discretize_state(
                current_row['avg_quiz_score'],
                current_row['completion_rate'],
                current_row['quiz_passed'],
                current_row['cluster'],
                current_row['sectionid']
            )
            
            action = get_action_from_type(next_row['type'])
            
            next_state = discretize_state(
                next_row['avg_quiz_score'],
                next_row['completion_rate'],
                next_row['quiz_passed'],
                next_row['cluster'],
                next_row['sectionid']
            )

            reward = get_reward(state, action, next_state)
            update_q_table(state, action, reward, next_state)
    print("✅ Huấn luyện Q-learning hoàn tất.")
    save_q_table_to_csv(q_table, 'q_table_results.csv')


# --- Hàm gợi ý hành động tốt nhất ---
def suggest_next_action(current_state, q_table):
    """
    Gợi ý hành động tốt nhất dựa trên trạng thái hiện tại và Q-table.
    """
    if current_state not in q_table:
        return random.choice(ACTIONS), 0.0
    
    q_values = q_table[current_state]
    best_action = max(q_values, key=q_values.get)
    best_q_value = q_values[best_action]
    
    return best_action, best_q_value

def track_and_suggest_new_rows(log_file_path, poll_interval=2):
    """
    Theo dõi file log mới, tính toán trạng thái, cập nhật Q-table và đưa ra gợi ý hành động.
    """
    global q_table
    print(f"\n🔍 Đang theo dõi file log: {log_file_path} và gợi ý hành động...")
    seen_lines = 0
    output_path = './user_insight.csv'

    if not os.path.exists(output_path):
        with open(output_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                'userid', 'courseid', 'sectionid', 'type',
                'avg_quiz_score', 'completion_rate', 'quiz_passed',
                'cluster', 'time'
            ])

    while True:
        try:
            if not os.path.exists(log_file_path):
                print("⚠️ File log chưa tồn tại. Đợi...")
                time.sleep(poll_interval)
                continue
            
            df_log = pd.read_csv(log_file_path)
            current_lines = len(df_log)

            if current_lines > seen_lines:
                new_rows = df_log.iloc[seen_lines:]
                print(f"\n🆕 Phát hiện {len(new_rows)} dòng log mới:")
                seen_lines = current_lines

                for _, row in new_rows.iterrows():
                    userid = int(row['userid'])
                    courseid = int(row['courseid'])
                    sectionid = int(row['sectionid'])
                    typ = str(row['type'])
                    timestamp = int(row['time'])

                    print(f"\n--- Xử lý hành động mới của User {userid} ({typ}) tại Section {sectionid} ---")

                    avg_quiz_score_data = call_api('local_userlog_get_avg_quiz_score', {
                        'userid': userid, 'courseid': courseid
                    })
                    avg_quiz_score = avg_quiz_score_data.get('avg_quiz_score', 0.0)

                    total_resource_data = call_api('local_userlog_get_total_resources_by_section', {
                        'sectionid': sectionid,
                        'objecttypes[0]': 'resource',
                        'objecttypes[1]': 'hvp',
                    })
                    total_resource = total_resource_data.get('total_resources', 0)

                    viewed_resource_data = call_api('local_userlog_get_viewed_resources_distinct_by_section', {
                        'userid': userid,
                        'courseid': courseid,
                        'sectionid': sectionid,
                        'objecttypes[0]': 'resource',
                        'objecttypes[1]': 'hvp',
                    })
                    viewed_resource = viewed_resource_data.get('viewed_resources', 0)

                    completion_rate = viewed_resource / total_resource if total_resource > 0 else 0

                    quiz_passed_data = call_api('local_userlog_get_latest_quiz_pass_status_by_section', {
                        'userid': userid,
                        'sectionid': sectionid,
                    })
                    quiz_passed = quiz_passed_data.get('is_passed', 0)

                    cluster = get_user_cluster(userid)

                    with open(output_path, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            userid,
                            courseid,
                            sectionid,
                            typ,
                            avg_quiz_score,
                            round(completion_rate, 2),
                            int(quiz_passed),
                            cluster,
                            timestamp
                        ])

                    current_state = discretize_state(
                        avg_quiz_score,
                        completion_rate,
                        quiz_passed,
                        cluster,
                        sectionid
                    )

                    user_data = pd.read_csv(output_path)
                    user_data = user_data[user_data['userid'] == userid].sort_values(by='time')

                    if len(user_data) >= 2:
                        prev_row = user_data.iloc[-2]
                        prev_state = discretize_state(
                            prev_row['avg_quiz_score'],
                            prev_row['completion_rate'],
                            prev_row['quiz_passed'],
                            prev_row['cluster'],
                            prev_row['sectionid']
                        )
                        prev_action = get_action_from_type(prev_row['type'])

                        reward = get_reward(prev_state, prev_action, current_state)
                        update_q_table(prev_state, prev_action, reward, current_state)

                        save_q_table_to_csv(q_table, 'q_table_results.csv')
                        print("💾 Q-table đã được cập nhật và lưu.")
                    
                    best_action, q_value = suggest_next_action(current_state, q_table)
                    print(f"👉 Trạng thái hiện tại của User {userid}: {current_state}")
                    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    print(f"💡 Đề xuất cho Section {sectionid}: '{best_action}' (Q-value = {q_value:.2f}) 🕒 {current_time}")

            time.sleep(poll_interval)

        except pd.errors.EmptyDataError:
            print("⚠️ File log rỗng. Đợi dữ liệu mới...")
            time.sleep(poll_interval)
        except Exception as e:
            print(f"❌ Lỗi: {e}")
            time.sleep(poll_interval)

# --- Quy trình chạy chính ---
if __name__ == "__main__":
    q_table_filename = 'q_table_results.csv'
    
    # 1. Khởi tạo Q-table nếu chưa tồn tại
    if not os.path.exists(q_table_filename):
        initialize_q_table_with_clusters(q_table_filename)

    # 2. Tải Q-table từ file
    q_table = load_q_table_from_csv(q_table_filename)

    # 3. Huấn luyện mô hình từ dữ liệu lịch sử
    # Hàm train_from_csv đã được sửa để tự động lưu lại kết quả
    train_from_csv('user_insight.csv')
    
    # 4. Tải lại Q-table đã được huấn luyện
    q_table = load_q_table_from_csv(q_table_filename)

    # 5. Bắt đầu theo dõi file log và đưa ra gợi ý thời gian thực
    log_file_path = "/Users/nguyenhuuloc/Documents/MyComputer/moodledata/local_userlog_data/user_log_summary.csv"
    track_and_suggest_new_rows(log_file_path)

✅ Q-table đã được tải từ file 'q_table_results.csv' thành công.
🚀 Bắt đầu huấn luyện mô hình Q-learning...
⚙️ Tính reward cho hành động 'attempt_new_quiz':
  Current state: (0, 0, 0, 0, 42)
  Next state: (1, 9, 1, 0, 42)
  +10: Quiz passed (từ 0 -> 1)
  +5: Completion rate tăng (0 -> 9)
  +7: Quiz score bin tăng (0 -> 1)
👉 Tổng reward: 22

⚙️ Tính reward cho hành động 'attempt_new_quiz':
  Current state: (1, 9, 1, 0, 42)
  Next state: (1, 9, 1, 0, 42)
👉 Tổng reward: 0

⚙️ Tính reward cho hành động 'attempt_new_quiz':
  Current state: (1, 9, 1, 0, 42)
  Next state: (1, 9, 1, 0, 42)
👉 Tổng reward: 0

⚙️ Tính reward cho hành động 'attempt_new_quiz':
  Current state: (1, 9, 1, 0, 42)
  Next state: (1, 9, 1, 0, 42)
👉 Tổng reward: 0

⚙️ Tính reward cho hành động 'attempt_new_quiz':
  Current state: (1, 9, 1, 0, 42)
  Next state: (1, 9, 1, 0, 42)
👉 Tổng reward: 0

⚙️ Tính reward cho hành động 'attempt_new_quiz':
  Current state: (1, 9, 1, 0, 42)
  Next state: (1, 9, 1, 0, 42)
👉 Tổng reward: 0

KeyboardInterrupt: 